In [46]:
import requests
import pandas as pd
import numpy as np 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### Four Square Credentials

In [52]:
Client_ID = 'WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF' # your Foursquare ID
Client_SECRET = 'XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + Client_ID)
print('Foursquare_Secret:' + Client_SECRET)

Your credentails:
Foursquare_ID: WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF
Foursquare_Secret:XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J


### Obtaining the particular city's longitude and latitude

In [53]:
city = 'Bangalore'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

12.9791198 77.5912997


### Hotel

In [54]:
search_query = 'Hotel'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(Client_ID, Client_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Hotel&radius=500&limit=30'

In [55]:
results = requests.get(url).json()


In [56]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ff931735', 'name': 'C...",False,53940333498e8a555ae0e2f3,NaN,IN,Bangalore,India,468,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98220710849808...",12.982207,77.594229,Vasanth Nagar,NaN,Karnātaka,Capitol Hotel,v-1566392436
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4f3bae47e4b04caf00fa4164,NaN,IN,Bangalore,India,461,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98209185115043...",12.982092,77.594259,NaN,NaN,Karnātaka,Hotel Paraag,v-1566392436
2,[],False,4de3d2418877bcb68664fc01,Bandra Kurla Complex,IN,Mumbai,India,513,"[Bandra Kurla Complex, Mumbai, Maharastra, India]","[{'label': 'display', 'lat': 12.983711, 'lng':...",12.983711,77.591756,NaN,NaN,Maharastra,Trident Hotel,v-1566392436
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bc7779f93bdeee1a66a37ae,"3, Raj Bhavan Rd",IN,Bangalore,India,476,"[3, Raj Bhavan Rd, Bangalore 560001, Karnātaka...","[{'label': 'display', 'lat': 12.98198012398084...",12.981980,77.594564,NaN,560001,Karnātaka,24 Carats,v-1566392436


In [57]:
# clean hotel data frame
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Capitol Hotel,Convention Center,NaN,IN,Bangalore,India,468,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98220710849808...",12.982207,77.594229,Vasanth Nagar,NaN,Karnātaka,53940333498e8a555ae0e2f3
1,Hotel Paraag,Hotel,NaN,IN,Bangalore,India,461,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98209185115043...",12.982092,77.594259,NaN,NaN,Karnātaka,4f3bae47e4b04caf00fa4164
2,Trident Hotel,None,Bandra Kurla Complex,IN,Mumbai,India,513,"[Bandra Kurla Complex, Mumbai, Maharastra, India]","[{'label': 'display', 'lat': 12.983711, 'lng':...",12.983711,77.591756,NaN,NaN,Maharastra,4de3d2418877bcb68664fc01
3,24 Carats,Hotel,"3, Raj Bhavan Rd",IN,Bangalore,India,476,"[3, Raj Bhavan Rd, Bangalore 560001, Karnātaka...","[{'label': 'display', 'lat': 12.98198012398084...",12.981980,77.594564,NaN,560001,Karnātaka,4bc7779f93bdeee1a66a37ae


In [58]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Capitol Hotel,Convention Center,NaN,12.982207,77.594229,NaN,Karnātaka
1,Hotel Paraag,Hotel,NaN,12.982092,77.594259,NaN,Karnātaka
2,Trident Hotel,None,Bandra Kurla Complex,12.983711,77.591756,NaN,Maharastra
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.981980,77.594564,560001,Karnātaka


In [59]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


In [60]:
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


In [61]:
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
3,24 Carats,Hotel,"3, Raj Bhavan Rd",12.98198,77.594564,560001,Karnātaka


### Parks

In [62]:
search_query = 'Park'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(Client_ID, Client_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Park&radius=10000&limit=30'

In [63]:
presults = requests.get(url).json()


In [64]:
venues = presults['response']['venues']

# tranform venues into a dataframe
park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bbc4513e45295217db855a4,M G Road,IN,Bangalore,India,NaN,489,"[M G Road, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97704217520378...",12.977042,77.595277,NaN,Karnātaka,Cubbon Park,v-1566392445
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ca83e2ca6e08cfa26708d94,Near Anand Rao Circle,IN,Bangalore,India,"5/43 , Race Course Road",1657,"[Near Anand Rao Circle (5/43 , Race Course Roa...","[{'label': 'display', 'lat': 12.98124239713509...",12.981242,77.576172,NaN,Karnātaka,Fortune Park JP Celestial,v-1566392445
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4f105c6fe4b072565b7811fa,NaN,IN,NaN,India,NaN,2042,[India],"[{'label': 'display', 'lat': 12.99253763031361...",12.992538,77.578450,NaN,NaN,Kumara Park West Park,v-1566392445
3,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4d88a92526a36ea88c04d0ad,"4/1, Bannerghatta Road",IN,Bangalore,India,NaN,5304,"[4/1, Bannerghatta Road, Bangalore 560029, Kar...","[{'label': 'display', 'lat': 12.93283615034002...",12.932836,77.602914,560029,Karnātaka,IBC Knowledge Park,v-1566392445
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4d181689512f2c0f952036cc,Rest House Road,IN,Bangalore,India,NaN,1659,"[Rest House Road, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97269293084298...",12.972693,77.605104,560001,Karnātaka,Rest House Park,v-1566392445


In [65]:
# Clean parks
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Cubbon Park,Park,M G Road,IN,Bangalore,India,NaN,489,"[M G Road, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97704217520378...",12.977042,77.595277,NaN,Karnātaka,4bbc4513e45295217db855a4
1,Fortune Park JP Celestial,Hotel,Near Anand Rao Circle,IN,Bangalore,India,"5/43 , Race Course Road",1657,"[Near Anand Rao Circle (5/43 , Race Course Roa...","[{'label': 'display', 'lat': 12.98124239713509...",12.981242,77.576172,NaN,Karnātaka,4ca83e2ca6e08cfa26708d94
2,Kumara Park West Park,Park,NaN,IN,NaN,India,NaN,2042,[India],"[{'label': 'display', 'lat': 12.99253763031361...",12.992538,77.578450,NaN,NaN,4f105c6fe4b072565b7811fa
3,IBC Knowledge Park,Building,"4/1, Bannerghatta Road",IN,Bangalore,India,NaN,5304,"[4/1, Bannerghatta Road, Bangalore 560029, Kar...","[{'label': 'display', 'lat': 12.93283615034002...",12.932836,77.602914,560029,Karnātaka,4d88a92526a36ea88c04d0ad
4,Rest House Park,Park,Rest House Road,IN,Bangalore,India,NaN,1659,"[Rest House Road, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97269293084298...",12.972693,77.605104,560001,Karnātaka,4d181689512f2c0f952036cc


In [66]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Cubbon Park,Park,M G Road,12.977042,77.595277,NaN,Karnātaka
1,Fortune Park JP Celestial,Hotel,Near Anand Rao Circle,12.981242,77.576172,NaN,Karnātaka
2,Kumara Park West Park,Park,NaN,12.992538,77.578450,NaN,NaN
3,IBC Knowledge Park,Building,"4/1, Bannerghatta Road",12.932836,77.602914,560029,Karnātaka
4,Rest House Park,Park,Rest House Road,12.972693,77.605104,560001,Karnātaka
5,Cubbon Park- Dog Park,Dog Run,NaN,12.974515,77.590203,NaN,NaN
6,Eden Park Restaurant,Indian Restaurant,"Next to Hindu Office, Off Infantry Road",12.983456,77.598707,560052,Karnātaka
7,Hotel Park Residency,Bed & Breakfast,NaN,12.972849,77.608019,NaN,NaN
8,The Park Hotel,Hotel,14/7 Mahatma Gandhi Road,12.973739,77.619766,NaN,Karnātaka
9,Manyata Embassy Business Park,Corporate Amenity,"Outer Ring Road, Nagavara",13.041540,77.619803,560 045,Karnātaka


In [67]:
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng,postalCode,state
3,IBC Knowledge Park,Building,"4/1, Bannerghatta Road",12.932836,77.602914,560029,Karnātaka
4,Rest House Park,Park,Rest House Road,12.972693,77.605104,560001,Karnātaka
6,Eden Park Restaurant,Indian Restaurant,"Next to Hindu Office, Off Infantry Road",12.983456,77.598707,560052,Karnātaka
9,Manyata Embassy Business Park,Corporate Amenity,"Outer Ring Road, Nagavara",13.041540,77.619803,560 045,Karnātaka
13,Bagmane Tech Park,Coworking Space,CV Raman Nagar,12.978464,77.660980,560093,Karnātaka
19,Bruhat Bangalore Mahanagar Palike Park,Park,Indiranagar,12.982505,77.594327,560038,Karnātaka
25,Freedom Park,Park,Seshadri Road,12.978008,77.582373,560001,Karnātaka
26,park road,Tech Startup,"Karnataka Royal Candles, Bengaluru, Karnataka ...",12.984553,77.599648,560051,Karnātaka
27,Prestige Park Square,Real Estate Office,"Sy. No. 25, Bannerghatta Road, Near NICE Road ...",12.972358,77.594150,560083,Karnātaka
29,Cariappa Park,Park,M G Road,12.976940,77.602032,560001,Karnātaka


In [68]:
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng,postalCode,state
4,Rest House Park,Park,Rest House Road,12.972693,77.605104,560001,Karnātaka
19,Bruhat Bangalore Mahanagar Palike Park,Park,Indiranagar,12.982505,77.594327,560038,Karnātaka
25,Freedom Park,Park,Seshadri Road,12.978008,77.582373,560001,Karnātaka
29,Cariappa Park,Park,M G Road,12.976940,77.602032,560001,Karnātaka


### Restaurants

In [69]:
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(Client_ID, Client_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Restaurant&radius=10000&limit=30'

In [70]:
Rresults = requests.get(url).json()


In [71]:
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4b6ee375f964a52029cf2ce3,"402-A, Millers Road, Vasanthnagar,",IN,Bangalore,India,NaN,1504,"[402-A, Millers Road, Vasanthnagar,, Bangalore...","[{'label': 'display', 'lat': 12.99227960861634...",12.992280,77.594473,560052,Karnātaka,Ujwal Bar & Restaurant,v-1566392455
1,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4d71c7d495bc8eec74015ff9,"173/1, S.C. Road,",IN,Bangalore,India,Seshadripuram,2313,"[173/1, S.C. Road, (Seshadripuram), Bangalore ...","[{'label': 'display', 'lat': 12.98939463563121...",12.989395,77.572764,560020,Karnātaka,New Samrat Restaurant,v-1566392455
2,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4f12e573e4b06c588c0319a3,NaN,IN,Bangalore,India,NaN,2198,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98887515906547...",12.988875,77.573682,NaN,Karnātaka,Uttarkhand Restaurant,v-1566392455
3,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4c00df159cf52d7f026c14e7,Church Street,IN,Bangalore,India,NaN,1925,"[Church Street, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97799896908608...",12.977999,77.609015,NaN,Karnātaka,RR Restaurant,v-1566392455
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4c793a984dc1ef3bd7fa2c9e,"Chalukya Hotel, Racecourse Road",IN,Bangalore,India,NaN,722,"[Chalukya Hotel, Racecourse Road, Bangalore, K...","[{'label': 'display', 'lat': 12.98463439463957...",12.984634,77.587782,NaN,Karnātaka,Samrat Restaurant,v-1566392455


In [72]:
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Ujwal Bar & Restaurant,Indian Restaurant,"402-A, Millers Road, Vasanthnagar,",IN,Bangalore,India,NaN,1504,"[402-A, Millers Road, Vasanthnagar,, Bangalore...","[{'label': 'display', 'lat': 12.99227960861634...",12.992280,77.594473,560052,Karnātaka,4b6ee375f964a52029cf2ce3
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",IN,Bangalore,India,Seshadripuram,2313,"[173/1, S.C. Road, (Seshadripuram), Bangalore ...","[{'label': 'display', 'lat': 12.98939463563121...",12.989395,77.572764,560020,Karnātaka,4d71c7d495bc8eec74015ff9
2,Uttarkhand Restaurant,Indian Restaurant,NaN,IN,Bangalore,India,NaN,2198,"[Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.98887515906547...",12.988875,77.573682,NaN,Karnātaka,4f12e573e4b06c588c0319a3
3,RR Restaurant,Indian Restaurant,Church Street,IN,Bangalore,India,NaN,1925,"[Church Street, Bangalore, Karnātaka, India]","[{'label': 'display', 'lat': 12.97799896908608...",12.977999,77.609015,NaN,Karnātaka,4c00df159cf52d7f026c14e7
4,Samrat Restaurant,Indian Restaurant,"Chalukya Hotel, Racecourse Road",IN,Bangalore,India,NaN,722,"[Chalukya Hotel, Racecourse Road, Bangalore, K...","[{'label': 'display', 'lat': 12.98463439463957...",12.984634,77.587782,NaN,Karnātaka,4c793a984dc1ef3bd7fa2c9e


In [73]:
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,crossStreet,lat,lng,postalCode,state
0,Ujwal Bar & Restaurant,Indian Restaurant,"402-A, Millers Road, Vasanthnagar,",NaN,12.992280,77.594473,560052,Karnātaka
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",Seshadripuram,12.989395,77.572764,560020,Karnātaka
2,Uttarkhand Restaurant,Indian Restaurant,NaN,NaN,12.988875,77.573682,NaN,Karnātaka
3,RR Restaurant,Indian Restaurant,Church Street,NaN,12.977999,77.609015,NaN,Karnātaka
4,Samrat Restaurant,Indian Restaurant,"Chalukya Hotel, Racecourse Road",NaN,12.984634,77.587782,NaN,Karnātaka
5,Eden Park Restaurant,Indian Restaurant,"Next to Hindu Office, Off Infantry Road",NaN,12.983456,77.598707,560052,Karnātaka
6,Mayuri Restaurant,Indian Restaurant,J C Road,NaN,12.956120,77.580845,560002,Karnātaka
7,Aathithya Restaurant,None,149 Infantry Road,NaN,12.983956,77.593435,NaN,Karnataka State
8,Imperial Restaurant,Indian Restaurant,Millers Road,NaN,12.991150,77.593837,NaN,Karnātaka
9,Nisarga Garden Restaurant,Indian Restaurant,Infantry Road,Opp. Commissioner's Office,12.983332,77.594431,560001,Karnātaka


In [74]:
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,crossStreet,lat,lng,postalCode,state
1,New Samrat Restaurant,Indian Restaurant,"173/1, S.C. Road,",Seshadripuram,12.989395,77.572764,560020,Karnātaka
9,Nisarga Garden Restaurant,Indian Restaurant,Infantry Road,Opp. Commissioner's Office,12.983332,77.594431,560001,Karnātaka
18,Hotel Shalimar Bar and Restaurant,Bar,322/1 CMH Road,Indiranagar,12.978253,77.637368,560038,Karnātaka
21,All Rounder Restaurant,Chinese Restaurant,KSCA,MG Road,12.979494,77.599570,500001,Karnātaka
25,Savoury Restaurant,Middle Eastern Restaurant,Mosque road,Frazer Town,12.999504,77.615887,560005,Karnātaka
26,Imperial Restaurant,Indian Restaurant,"#42/5, Opp. Shivajinagar Bus Stand",Central Street,12.982176,77.603279,560005,Karnātaka
29,Grand Spicy Restaurant,Indian Restaurant,"#9, Mota Chambers",Millers Road,12.990051,77.593160,560052,Karnātaka


### Cafeteria

In [75]:
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(Client_ID, Client_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [76]:
cresults = requests.get(url).json()


In [77]:
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4dd4b6bd183859a589a94fc7,"10th Floor, Purva Premier",IN,Bangalore,India,Residency Road,1686,"[10th Floor, Purva Premier (Residency Road), B...","[{'label': 'display', 'lat': 12.96601946504236...",12.966019,77.599111,560025,Karnātaka,Unisys Cafeteria,v-1566392467
1,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,4cfde38fc6cca35d463e9932,Mahadevapura,IN,Bangalore,India,KR Puram-Marathahalli Ring Road,11151,[Mahadevapura (KR Puram-Marathahalli Ring Road...,"[{'label': 'display', 'lat': 12.98172589173957...",12.981726,77.694068,560048,Karnātaka,EMC India Cafeteria,v-1566392467
2,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,5045ac7de4b0930467826757,NaN,IN,NaN,India,NaN,2105,[India],"[{'label': 'display', 'lat': 12.96137577734386...",12.961376,77.598020,NaN,NaN,Cafeteria,v-1566392467
3,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4f0d4265e4b07f0a56ebd92d,St. Joseph College Of Commerce,IN,Bangalore,India,Brigade Road,1964,[St. Joseph College Of Commerce (Brigade Road)...,"[{'label': 'display', 'lat': 12.97026778533008...",12.970268,77.606961,560025,Karnātaka,Cafeteria @ SJCC,v-1566392467
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,5370ad13498e080bda024bd4,"No 30, 10th Main, Ashoka Pillar Road",IN,Bangalore,India,Jayanagar 2nd Block,4453,"[No 30, 10th Main, Ashoka Pillar Road (Jayanag...","[{'label': 'display', 'lat': 12.93959043338276...",12.939590,77.584987,560068,Karnātaka,British Biologicals Cafeteria,v-1566392467


In [78]:
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Unisys Cafeteria,Food Court,"10th Floor, Purva Premier",IN,Bangalore,India,Residency Road,1686,"[10th Floor, Purva Premier (Residency Road), B...","[{'label': 'display', 'lat': 12.96601946504236...",12.966019,77.599111,560025,Karnātaka,4dd4b6bd183859a589a94fc7
1,EMC India Cafeteria,Cafeteria,Mahadevapura,IN,Bangalore,India,KR Puram-Marathahalli Ring Road,11151,[Mahadevapura (KR Puram-Marathahalli Ring Road...,"[{'label': 'display', 'lat': 12.98172589173957...",12.981726,77.694068,560048,Karnātaka,4cfde38fc6cca35d463e9932
2,Cafeteria,Café,NaN,IN,NaN,India,NaN,2105,[India],"[{'label': 'display', 'lat': 12.96137577734386...",12.961376,77.598020,NaN,NaN,5045ac7de4b0930467826757
3,Cafeteria @ SJCC,College Cafeteria,St. Joseph College Of Commerce,IN,Bangalore,India,Brigade Road,1964,[St. Joseph College Of Commerce (Brigade Road)...,"[{'label': 'display', 'lat': 12.97026778533008...",12.970268,77.606961,560025,Karnātaka,4f0d4265e4b07f0a56ebd92d
4,British Biologicals Cafeteria,Office,"No 30, 10th Main, Ashoka Pillar Road",IN,Bangalore,India,Jayanagar 2nd Block,4453,"[No 30, 10th Main, Ashoka Pillar Road (Jayanag...","[{'label': 'display', 'lat': 12.93959043338276...",12.939590,77.584987,560068,Karnātaka,5370ad13498e080bda024bd4


In [79]:
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Unisys Cafeteria,Food Court,"10th Floor, Purva Premier",12.966019,77.599111,560025,Karnātaka
1,EMC India Cafeteria,Cafeteria,Mahadevapura,12.981726,77.694068,560048,Karnātaka
2,Cafeteria,Café,NaN,12.961376,77.598020,NaN,NaN
3,Cafeteria @ SJCC,College Cafeteria,St. Joseph College Of Commerce,12.970268,77.606961,560025,Karnātaka
4,British Biologicals Cafeteria,Office,"No 30, 10th Main, Ashoka Pillar Road",12.939590,77.584987,560068,Karnātaka
5,Lexington Towers Cafeteria,Food Court,Oracle India Lexington Towers,12.930891,77.610290,560029,Karnātaka
6,Accenture Cafeteria,Food Court,Accenture Campus,12.991861,77.588803,NaN,Karnātaka
7,Cafeteria,College Cafeteria,Christ University (Auditorium Block),12.936183,77.606122,560029,Karnātaka
8,Cafeteria,Food Court,St. Joseph College Of Commerce,12.935831,77.615348,NaN,Karnātaka
9,Intel Security Cafeteria,Cafeteria,"I floor, Pine Valley, EGL Business Park",12.946524,77.646523,560071,Karnātaka


In [80]:
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,postalCode,state
0,Unisys Cafeteria,Food Court,"10th Floor, Purva Premier",12.966019,77.599111,560025,Karnātaka
1,EMC India Cafeteria,Cafeteria,Mahadevapura,12.981726,77.694068,560048,Karnātaka
3,Cafeteria @ SJCC,College Cafeteria,St. Joseph College Of Commerce,12.970268,77.606961,560025,Karnātaka
4,British Biologicals Cafeteria,Office,"No 30, 10th Main, Ashoka Pillar Road",12.939590,77.584987,560068,Karnātaka
5,Lexington Towers Cafeteria,Food Court,Oracle India Lexington Towers,12.930891,77.610290,560029,Karnātaka
7,Cafeteria,College Cafeteria,Christ University (Auditorium Block),12.936183,77.606122,560029,Karnātaka
9,Intel Security Cafeteria,Cafeteria,"I floor, Pine Valley, EGL Business Park",12.946524,77.646523,560071,Karnātaka
10,"Rooftop Cafeteria, Crystal Downs (Goldman Sachs)",Food Court,"Crystal Downs, Embassy Golf Links Business Par...",12.949784,77.644611,560071,Karnātaka
15,BIO Cafeteria,Cafeteria,Bangalore Institute of Oncology,12.960538,77.593045,560027,Karnātaka
20,GroupM Cafeteria.,Snack Place,"29, Mahalakshmi Chambers",12.973219,77.616259,560001,Karnātaka


### Shopping

In [81]:
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(Client_ID, Client_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WTYXTX1VKJMLXOJPLAD1IJKLASAMY20FDVGFA1Z1X4XTYAHF&client_secret=XV3U42TL5SPPKSLA4KYBIKK3JW3FRLEJDZJJNMQCXS31ZY5J&ll=12.9791198,77.5912997&v=20180604&query=Shopping&radius=1000&limit=30'

In [82]:
sresults = requests.get(url).json()
#sresults

In [83]:
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",False,5632ec65498eb65f4c43e98e,BatteryBhai.com,IN,Bangalore,India,305,"[BatteryBhai.com, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97846402705198...",12.978464,77.594032,560001,Karnātaka,Best Car Battery Online Shopping Store,v-1566392479


In [84]:
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Best Car Battery Online Shopping Store,Automotive Shop,BatteryBhai.com,IN,Bangalore,India,305,"[BatteryBhai.com, Bangalore 560001, Karnātaka,...","[{'label': 'display', 'lat': 12.97846402705198...",12.978464,77.594032,560001,Karnātaka,5632ec65498eb65f4c43e98e


In [85]:
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Best Car Battery Online Shopping Store,Automotive Shop,BatteryBhai.com,12.978464,77.594032,560001,Karnātaka


In [91]:
hotel_neighbourhood_df = pd.concat([df_hotels, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,24 Carats,Hotel,"3, Raj Bhavan Rd",12.981980,77.594564,560001,Karnātaka
1,Unisys Cafeteria,Food Court,"10th Floor, Purva Premier",12.966019,77.599111,560025,Karnātaka
2,EMC India Cafeteria,Cafeteria,Mahadevapura,12.981726,77.694068,560048,Karnātaka
3,Cafeteria @ SJCC,College Cafeteria,St. Joseph College Of Commerce,12.970268,77.606961,560025,Karnātaka
4,British Biologicals Cafeteria,Office,"No 30, 10th Main, Ashoka Pillar Road",12.939590,77.584987,560068,Karnātaka
5,Lexington Towers Cafeteria,Food Court,Oracle India Lexington Towers,12.930891,77.610290,560029,Karnātaka
6,Cafeteria,College Cafeteria,Christ University (Auditorium Block),12.936183,77.606122,560029,Karnātaka
7,Intel Security Cafeteria,Cafeteria,"I floor, Pine Valley, EGL Business Park",12.946524,77.646523,560071,Karnātaka
8,"Rooftop Cafeteria, Crystal Downs (Goldman Sachs)",Food Court,"Crystal Downs, Embassy Golf Links Business Par...",12.949784,77.644611,560071,Karnātaka
9,BIO Cafeteria,Cafeteria,Bangalore Institute of Oncology,12.960538,77.593045,560027,Karnātaka


### Mapping and Clustering

In [92]:
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map

In [94]:
park_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
park_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,address,categories,crossStreet,lat,lng,name,postalCode,state
0,Rest House Road,Park,NaN,12.972693,77.605104,Rest House Park,560001,Karnātaka
1,Indiranagar,Park,NaN,12.982505,77.594327,Bruhat Bangalore Mahanagar Palike Park,560038,Karnātaka
2,Seshadri Road,Park,NaN,12.978008,77.582373,Freedom Park,560001,Karnātaka
3,M G Road,Park,NaN,12.976940,77.602032,Cariappa Park,560001,Karnātaka
4,"173/1, S.C. Road,",Indian Restaurant,Seshadripuram,12.989395,77.572764,New Samrat Restaurant,560020,Karnātaka
5,Infantry Road,Indian Restaurant,Opp. Commissioner's Office,12.983332,77.594431,Nisarga Garden Restaurant,560001,Karnātaka
6,322/1 CMH Road,Bar,Indiranagar,12.978253,77.637368,Hotel Shalimar Bar and Restaurant,560038,Karnātaka
7,KSCA,Chinese Restaurant,MG Road,12.979494,77.599570,All Rounder Restaurant,500001,Karnātaka
8,Mosque road,Middle Eastern Restaurant,Frazer Town,12.999504,77.615887,Savoury Restaurant,560005,Karnātaka
9,"#42/5, Opp. Shivajinagar Bus Stand",Indian Restaurant,Central Street,12.982176,77.603279,Imperial Restaurant,560005,Karnātaka


In [95]:
park_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(park_neighbourhood_df['lat'], park_neighbourhood_df['lng'], 
                                           park_neighbourhood_df['name'], park_neighbourhood_df['categories'],\
                                               park_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(park_map)  
    
park_map

In [96]:
shop_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
shop_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,address,categories,crossStreet,lat,lng,name,postalCode,state
0,Rest House Road,Park,NaN,12.972693,77.605104,Rest House Park,560001,Karnātaka
1,Indiranagar,Park,NaN,12.982505,77.594327,Bruhat Bangalore Mahanagar Palike Park,560038,Karnātaka
2,Seshadri Road,Park,NaN,12.978008,77.582373,Freedom Park,560001,Karnātaka
3,M G Road,Park,NaN,12.976940,77.602032,Cariappa Park,560001,Karnātaka
4,"173/1, S.C. Road,",Indian Restaurant,Seshadripuram,12.989395,77.572764,New Samrat Restaurant,560020,Karnātaka
5,Infantry Road,Indian Restaurant,Opp. Commissioner's Office,12.983332,77.594431,Nisarga Garden Restaurant,560001,Karnātaka
6,322/1 CMH Road,Bar,Indiranagar,12.978253,77.637368,Hotel Shalimar Bar and Restaurant,560038,Karnātaka
7,KSCA,Chinese Restaurant,MG Road,12.979494,77.599570,All Rounder Restaurant,500001,Karnātaka
8,Mosque road,Middle Eastern Restaurant,Frazer Town,12.999504,77.615887,Savoury Restaurant,560005,Karnātaka
9,"#42/5, Opp. Shivajinagar Bus Stand",Indian Restaurant,Central Street,12.982176,77.603279,Imperial Restaurant,560005,Karnātaka


In [97]:
shop_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(shop_neighbourhood_df['lat'], shop_neighbourhood_df['lng'], 
                                           shop_neighbourhood_df['name'], shop_neighbourhood_df['categories'],\
                                               shop_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(shop_map)  
    
shop_map

In [98]:
cafe_neighbourhood_df = pd.concat([df_park, df_Restaurant, df_Cafeteria,], ignore_index=True)
cafe_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,address,categories,crossStreet,lat,lng,name,postalCode,state
0,Rest House Road,Park,NaN,12.972693,77.605104,Rest House Park,560001,Karnātaka
1,Indiranagar,Park,NaN,12.982505,77.594327,Bruhat Bangalore Mahanagar Palike Park,560038,Karnātaka
2,Seshadri Road,Park,NaN,12.978008,77.582373,Freedom Park,560001,Karnātaka
3,M G Road,Park,NaN,12.976940,77.602032,Cariappa Park,560001,Karnātaka
4,"173/1, S.C. Road,",Indian Restaurant,Seshadripuram,12.989395,77.572764,New Samrat Restaurant,560020,Karnātaka
5,Infantry Road,Indian Restaurant,Opp. Commissioner's Office,12.983332,77.594431,Nisarga Garden Restaurant,560001,Karnātaka
6,322/1 CMH Road,Bar,Indiranagar,12.978253,77.637368,Hotel Shalimar Bar and Restaurant,560038,Karnātaka
7,KSCA,Chinese Restaurant,MG Road,12.979494,77.599570,All Rounder Restaurant,500001,Karnātaka
8,Mosque road,Middle Eastern Restaurant,Frazer Town,12.999504,77.615887,Savoury Restaurant,560005,Karnātaka
9,"#42/5, Opp. Shivajinagar Bus Stand",Indian Restaurant,Central Street,12.982176,77.603279,Imperial Restaurant,560005,Karnātaka


In [99]:
cafe_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(cafe_neighbourhood_df['lat'], cafe_neighbourhood_df['lng'], 
                                           cafe_neighbourhood_df['name'], cafe_neighbourhood_df['categories'],\
                                               cafe_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(cafe_map)  
    
cafe_map